In [114]:
# Requires numpy, pandas, wrapt, scipy, networkx 1.11
import math
import itertools
import numpy as np
import copy, pprint

from pgmpy.models import BayesianModel, ClusterGraph, MarkovModel
from pgmpy.factors.discrete import TabularCPD, DiscreteFactor
from pgmpy.inference.ExactInference import BeliefPropagation

# NOTE: np.random.rand is sampled from a uniform distribution [0,1)
np.random.seed(seed=1)

In [115]:
intervals = [(1,100),(1,200),(50,150),(50,100)]

factor_A = DiscreteFactor(variables=["A"], cardinality=[2],
                          values=np.random.uniform(intervals[0][0], intervals[0][1],size=(2)))
factor_B = DiscreteFactor(variables=["B"], cardinality=[4],
                          values=np.random.uniform(intervals[1][0], intervals[1][1],size=(4)))
factor_C = DiscreteFactor(variables=["C"], cardinality=[2],
                          values=np.random.uniform(intervals[2][0], intervals[2][1],size=(2)))
factor_D = DiscreteFactor(variables=["D"], cardinality=[2],
                          values=np.random.uniform(intervals[3][0], intervals[3][1],size=(2)))
factor_E = DiscreteFactor(variables=["E"], cardinality=[2],
                          values=np.random.uniform(intervals[0][0], intervals[0][1],size=(2)))

factor_AB = DiscreteFactor(variables=["A", "B"], cardinality=[2,4],
                           values=np.random.uniform(intervals[0][0], intervals[0][1],size=(2,4)))
factor_BCD = DiscreteFactor(variables=["B", "C", "D"], cardinality=[4,2,2],
                            values=np.random.uniform(intervals[1][0], intervals[1][1],size=(4,2,2)))
factor_AD = DiscreteFactor(variables=["A", "D"], cardinality=[2,2],
                           values=np.random.uniform(intervals[2][0], intervals[2][1],size=(2,2)))
factor_CE = DiscreteFactor(variables=["C", "E"], cardinality=[2,2],
                           values=np.random.uniform(intervals[3][0], intervals[3][1],size=(2,2)))

my_factors = [factor_A, factor_B, factor_C, factor_D, factor_E,
              factor_AB, factor_BCD, factor_AD, factor_CE]

MM = MarkovModel([("A","B"), ("B","C"),("B","D"), ("C","D"), ("A","D"),("C","E")])
MM.add_factors(*my_factors)
print(MM.nodes())
print(MM.edges())
print(MM.check_model())

['E', 'A', 'D', 'B', 'C']
[('E', 'C'), ('A', 'B'), ('A', 'D'), ('D', 'B'), ('D', 'C'), ('B', 'C')]
True


In [116]:
for f in my_factors:
    print(f)

╒═════╤══════════╕
│ A   │   phi(A) │
╞═════╪══════════╡
│ A_0 │  42.2852 │
├─────┼──────────┤
│ A_1 │  72.3121 │
╘═════╧══════════╛
╒═════╤══════════╕
│ B   │   phi(B) │
╞═════╪══════════╡
│ B_0 │   1.0228 │
├─────┼──────────┤
│ B_1 │  61.1642 │
├─────┼──────────┤
│ B_2 │  30.2044 │
├─────┼──────────┤
│ B_3 │  19.3754 │
╘═════╧══════════╛
╒═════╤══════════╕
│ C   │   phi(C) │
╞═════╪══════════╡
│ C_0 │  68.6260 │
├─────┼──────────┤
│ C_1 │  84.5561 │
╘═════╧══════════╛
╒═════╤══════════╕
│ D   │   phi(D) │
╞═════╪══════════╡
│ D_0 │  69.8384 │
├─────┼──────────┤
│ D_1 │  76.9408 │
╘═════╧══════════╛
╒═════╤══════════╕
│ E   │   phi(E) │
╞═════╪══════════╡
│ E_0 │  42.5003 │
├─────┼──────────┤
│ E_1 │  68.8367 │
╘═════╧══════════╛
╒═════╤═════╤════════════╕
│ A   │ B   │   phi(A,B) │
╞═════╪═════╪════════════╡
│ A_0 │ B_0 │    21.2408 │
├─────┼─────┼────────────┤
│ A_0 │ B_1 │    87.9336 │
├─────┼─────┼────────────┤
│ A_0 │ B_2 │     3.7114 │
├─────┼─────┼────────────┤
│ A_0 │ B_3 │   

In [118]:
def create_bethe_cluster_graph(factors):
    bethe_cg = ClusterGraph()
    uni = []
    multi = []
    for f in factors:
        if len(f.scope()) == 1:
            uni.append(f.scope())
        else:
            multi.append(f.scope())
    
    for mf in multi:
        for uf in uni:
            if uf[0] in mf:
                bethe_cg.add_edge(uf[0], tuple(mf))
                
    bethe_cg.add_factors(*factors)
    
    # factor list order should NOT be changed!
    alpha = {}
    for node, i in zip(bethe_cg.nodes(), range(len(factors))):
        alpha[i] = node
    
    return bethe_cg, alpha

In [119]:
cg, alpha = create_bethe_cluster_graph(my_factors)

In [120]:
pprint.pprint(vars(cg))

{'adj': {'A': {('A', 'B'): {'weight': None}, ('A', 'D'): {'weight': None}},
         'B': {('A', 'B'): {'weight': None}, ('B', 'C', 'D'): {'weight': None}},
         'C': {('B', 'C', 'D'): {'weight': None}, ('C', 'E'): {'weight': None}},
         'D': {('A', 'D'): {'weight': None}, ('B', 'C', 'D'): {'weight': None}},
         'E': {('C', 'E'): {'weight': None}},
         ('A', 'B'): {'A': {'weight': None}, 'B': {'weight': None}},
         ('A', 'D'): {'A': {'weight': None}, 'D': {'weight': None}},
         ('B', 'C', 'D'): {'B': {'weight': None},
                           'C': {'weight': None},
                           'D': {'weight': None}},
         ('C', 'E'): {'C': {'weight': None}, 'E': {'weight': None}}},
 'adjlist_dict_factory': <class 'dict'>,
 'edge': {'A': {('A', 'B'): {'weight': None}, ('A', 'D'): {'weight': None}},
          'B': {('A', 'B'): {'weight': None},
                ('B', 'C', 'D'): {'weight': None}},
          'C': {('B', 'C', 'D'): {'weight': None},
         

In [122]:
def init_cluster_graph(cluster_graph):
    cluster_beliefs = {}
    sepset_beliefs = {}
    
    for node in cluster_graph.nodes():
        init_belief = False
        for factor in [cluster_graph.get_factors(node)]:
            if init_belief == False:
                init_belief = factor.copy()
            else:
                init_belief *= factor.copy()
        cluster_beliefs[node] = init_belief.copy()
    
    for edge in cluster_graph.edges():
        sepset_beliefs[(edge[0], edge[1])] = 1
        sepset_beliefs[(edge[1], edge[0])] = 1
        
    return cluster_beliefs, sepset_beliefs

In [123]:
cb, sb = init_cluster_graph(cg)

In [125]:
def sum_product_message(cluster_graph, cluster_beliefs, sepset_beliefs, node_i, node_j):
    node_i_belief = cluster_beliefs[node_i].copy()
    for edge in cluster_graph.edges(node_i):
        if node_j not in edge:
            node_i_belief *= sepset_beliefs[edge]
    
    # We assume CG is a BCG
    var_diff = []
    if len(node_i) > len(node_j):
        cluster_scope = set(node_i)
        sepset_scope = set(node_j)
        var_diff = cluster_scope.difference(sepset_scope) 
        
    sepset_beliefs[(node_i,node_j)] = node_i_belief.marginalize(var_diff, inplace=False)

In [126]:
def LoopyBeliefPropagation(cluster_graph, num_iterations=1500):
    cluster_beliefs, sepset_beliefs = init_cluster_graph(cluster_graph)
    
    reverse_pass = {}
    for edge in cluster_graph.edges():
        reverse_pass[edge] = True
    
    curr_it = 0
    while True:
        has_converged = True
        curr_it += 1
        for edge in cluster_graph.edges():
            node_i = edge[0]
            node_j = edge[1]
            
            if len(node_i) > len(node_j):
                cluster_scope_i = set(node_i)
                sepset_scope_ij = set(node_j)
                diff_scope_i = cluster_scope_i.difference(sepset_scope_ij)
                diff_scope_j = []
            else:
                cluster_scope_j = set(node_j)
                sepset_scope_ij = set(node_i)
                diff_scope_j = cluster_scope_j.difference(sepset_scope_ij)
                diff_scope_i = []
            
            node_i_belief = cluster_beliefs[node_i].copy()
            for e in cluster_graph.edges(node_i):
                node_i_belief *= sepset_beliefs[e]
                
            node_j_belief = cluster_beliefs[node_j].copy()
            for e in cluster_graph.edges(node_j):
                node_j_belief *= sepset_beliefs[e]
            
            marg_belief_i = node_i_belief.marginalize(diff_scope_i, inplace=False)
            marg_belief_j = node_j_belief.marginalize(diff_scope_j, inplace=False)
            
            if np.allclose(marg_belief_i.values, marg_belief_j.values):
                pass
            else:
                has_converged = False
                if reverse_pass[edge]:
                    sum_product_message(cluster_graph, cluster_beliefs, sepset_beliefs, 
                                        node_i, node_j)
                    reverse_pass[edge] = False
                else:
                    sum_product_message(cluster_graph, cluster_beliefs, sepset_beliefs, 
                                        node_j, node_i)
                    reverse_pass[edge] = True

        if curr_it >= num_iterations or has_converged:
            break
        
    for node in cluster_graph.nodes():
        for edge in cluster_graph.edges(node):
            cluster_beliefs[node] *= sepset_beliefs[edge]
    
    return cluster_beliefs, sepset_beliefs

In [127]:
new_cb, new_sb = LoopyBeliefPropagation(my_factors, cg, num_iterations=2000)

/usr/local/lib/python3.5/dist-packages/pgmpy/factors/discrete/DiscreteFactor.py:610: RuntimeWarning: overflow encountered in multiply
  phi.values = phi.values * phi1.values


In [113]:
for c, val in new_cb.items():
    print(val)

╒═════╤═════╤════════════╕
│ A   │ D   │   phi(A,D) │
╞═════╪═════╪════════════╡
│ A_0 │ D_0 │        inf │
├─────┼─────┼────────────┤
│ A_0 │ D_1 │        inf │
├─────┼─────┼────────────┤
│ A_1 │ D_0 │        inf │
├─────┼─────┼────────────┤
│ A_1 │ D_1 │        inf │
╘═════╧═════╧════════════╛
╒═════╤══════════╕
│ C   │   phi(C) │
╞═════╪══════════╡
│ C_0 │      inf │
├─────┼──────────┤
│ C_1 │      inf │
╘═════╧══════════╛
╒═════╤══════════╕
│ D   │   phi(D) │
╞═════╪══════════╡
│ D_0 │      inf │
├─────┼──────────┤
│ D_1 │      inf │
╘═════╧══════════╛
╒═════╤══════════╕
│ E   │   phi(E) │
╞═════╪══════════╡
│ E_0 │  48.0506 │
├─────┼──────────┤
│ E_1 │ 169.3382 │
╘═════╧══════════╛
╒═════╤═════╤════════════╕
│ C   │ E   │   phi(C,E) │
╞═════╪═════╪════════════╡
│ C_0 │ E_0 │        inf │
├─────┼─────┼────────────┤
│ C_0 │ E_1 │        inf │
├─────┼─────┼────────────┤
│ C_1 │ E_0 │        inf │
├─────┼─────┼────────────┤
│ C_1 │ E_1 │        inf │
╘═════╧═════╧════════════╛
╒═════╤

In [90]:
for c, val in cb.items():
    print(val)

╒═════╤═════╤════════════╕
│ A   │ D   │   phi(A,D) │
╞═════╪═════╪════════════╡
│ A_0 │ D_0 │   110.6329 │
├─────┼─────┼────────────┤
│ A_0 │ D_1 │   106.8851 │
├─────┼─────┼────────────┤
│ A_1 │ D_0 │    81.7362 │
├─────┼─────┼────────────┤
│ A_1 │ D_1 │   148.8616 │
╘═════╧═════╧════════════╛
╒═════╤══════════╕
│ C   │   phi(C) │
╞═════╪══════════╡
│ C_0 │ 136.3542 │
├─────┼──────────┤
│ C_1 │ 124.7122 │
╘═════╧══════════╛
╒═════╤══════════╕
│ D   │   phi(D) │
╞═════╪══════════╡
│ D_0 │  77.8120 │
├─────┼──────────┤
│ D_1 │  56.8228 │
╘═════╧══════════╛
╒═════╤══════════╕
│ E   │   phi(E) │
╞═════╪══════════╡
│ E_0 │   6.9319 │
├─────┼──────────┤
│ E_1 │  13.0130 │
╘═════╧══════════╛
╒═════╤═════╤════════════╕
│ C   │ E   │   phi(C,E) │
╞═════╪═════╪════════════╡
│ C_0 │ E_0 │    78.9873 │
├─────┼─────┼────────────┤
│ C_0 │ E_1 │    69.0071 │
├─────┼─────┼────────────┤
│ C_1 │ E_0 │    77.5474 │
├─────┼─────┼────────────┤
│ C_1 │ E_1 │    87.2667 │
╘═════╧═════╧════════════╛
╒═════╤